In [7]:
import requests
import json
import re
import bs4
import time
import numpy as np
import pandas as pd

import sys
import signal

import CONFIG

In [8]:
def getBSFromURL(url):
    try:
        r = requests.get(url, headers = CONFIG.HEADER)
        return getBS(r.text)
    
    except requests.exceptions.RequestException as e:
        print("Connection Error")

def getBS(html):
    html_bs = bs4.BeautifulSoup(html, 'html.parser')
    return html_bs

## Get all Issues from selected decades

We scrape all the information of articles first from the main page of JSTOR.

By default, as in https://www.jstor.org/journal/amereconrevi, the titles of decades are first listed. Clicking on the decade makes an request to JSTOR backend to get the full list of the issues within the decade. 

The request are in the form of `https://www.jstor.org/journal/amereconrevi/decade/` + `ID`, where the `ID` is stored in the `data-filter` attribute in the element with class `expand-arrow`.

This returns a json object which contains the HTML element to append in the `ul` element, but we can simply turn them into `soup` objects and extract the link to the issues as well as the issue name, which are the `href` and the `text` of the `"a"` selector, respectively.

In [9]:
# a very simple sleeping function to avoid getting banned
def ImNotRobot():
    sleepTime = np.random.uniform(5,15)
    print(f"sleeping {sleepTime} seconds")
    time.sleep(sleepTime)

# see if list of issue file already exist
def allIssueListExist(path_to_file):
    from os.path import exists
    return exists(path_to_file)


# get url that returns all list if issues within the decade
def getURLinDecade(journal_url):
    
    journal_bs = getBSFromURL(journal_url)
    decade_informations = journal_bs.select(".expand-arrow")

    decade_issues_lists_url = [f"{journal_url}/decade/{u['data-filter']}" for u in decade_informations]
    return decade_issues_lists_url

# go over the urls and get the url for all issues

def handleIssueText(t):
    return t.split('pp')[0].strip()
    

    
def getAllIssuesInDecade(url):
    print ("processing " + url)
    issue_request = requests.get(url, headers = CONFIG.HEADER)

    issue_html = json.loads(issue_request.text)[0]
    soup = getBS(issue_html)
    all_issue = soup.select("a")
    
    print(all_issue)
    issues = []
    for issue in all_issue:
        issue_info = {
            'name':handleIssueText(issue.text),
            'url':issue['href']
        }
        issues.append(issue_info)
        
    return issues



In [15]:
if allIssueListExist( f"{CONFIG.JOURNAL_NAME}.csv"):
    loadAllIsues()
else:
    journal_url = CONFIG.URL
    URLinDecade = getURLinDecade(journal_url)
    
    allIsues = []
    for u in URLinDecade:
        ImNotRobot()
        allIsues += getAllIssuesInDecade(u)
        
with open('outputfile', 'w') as fout:
    json.dump(allIsues, fout)

sleeping 32.77096318873478 seconds
processing https://www.jstor.org/journal/amereconrevi/decade/AXllYXI6WzIwMTAgVE8gMjAyMH0
[<a href="/stable/10.2307/e26848476">No. 12 DECEMBER 2019  pp. 4071-4425</a>, <a href="/stable/10.2307/e26807864">No. 11 NOVEMBER 2019  pp. 3713-4070</a>, <a href="/stable/10.2307/e26789060">No. 10 OCTOBER 2019  pp. 3395-3712</a>, <a href="/stable/10.2307/e26773255">No. 9 September 2019  pp. 3031-3394</a>, <a href="/stable/10.2307/e26747403">No. 8 AUGUST 2019  pp. 2693-3029</a>, <a href="/stable/10.2307/e26739273">No. 7 JULY 2019  pp. 2383-2691</a>, <a href="/stable/10.2307/e26737876">No. 6 JUNE 2019  pp. 1991-2381</a>, <a href="/stable/10.2307/e26636969">No. 5 MAY 2019  pp. 1623-1990</a>, <a href="/stable/10.2307/e26637201">No. 4 APRIL 2019  pp. 1197-1621</a>, <a href="/stable/10.2307/e26602966">No. 3 MARCH 2019  pp. 116-1195</a>, <a href="/stable/10.2307/e26603145">No. 2 FEBRUARY 2019  pp. 325-737</a>, <a href="/stable/10.2307/e26562851">No. 1 JANUARY 2019  pp. 

processing https://www.jstor.org/journal/amereconrevi/decade/AXllYXI6WzIwMDAgVE8gMjAxMH0
[<a href="/stable/10.2307/i25592529">No. 5 Dec., 2009  pp. 1689-2277, i-xxxi</a>, <a href="/stable/10.2307/i25592499">No. 4 Sep., 2009  pp. 1097-1683, i-xvii</a>, <a href="/stable/10.2307/i25592471">No. 3 Jun., 2009  pp. 561-1091, i-ix</a>, <a href="/stable/10.2307/i25592364">No. 2 Papers and Proceedings of the One Hundred Twenty-First Meeting of the American Economic Association May, 2009  pp. i-x, 1-646</a>, <a href="/stable/10.2307/i29730173">No. 1 Mar., 2009  pp. 1-555, i-x</a>, <a href="/stable/10.2307/i29730147">No. 5 Dec., 2008  pp. 1737-2273, i-xxxiv</a>, <a href="/stable/10.2307/i29730117">No. 4 Sep., 2008  pp. 1211-1732, i-xviii</a>, <a href="/stable/10.2307/i29730083">No. 3 Jun., 2008  pp. 567-1205, i-xiii</a>, <a href="/stable/10.2307/i29729986">No. 2 Papers and Proceedings of the One Hundred Twentieth Annual Meeting of the American Economic Association May, 2008  pp. i-x, 1-634, i-xii<

processing https://www.jstor.org/journal/amereconrevi/decade/AXllYXI6WzE5ODAgVE8gMTk5MH0
[<a href="/stable/10.2307/i332843">No. 6 Nobel Lectures and 1989 Survey of Members Dec., 1989  pp. iv-v+vii-xi+1-654+i-xiv</a>, <a href="/stable/10.2307/i331916">No. 5 Dec., 1989  pp. 957-1307+i-xxxvi</a>, <a href="/stable/10.2307/i331802">No. 4 Sep., 1989  pp. 621-955+i-xliv</a>, <a href="/stable/10.2307/i304577">No. 3 Jun., 1989  pp. 295-620+i-xxii</a>, <a href="/stable/10.2307/i331803">No. 2 Papers and Proceedings of the Hundred and First Annual Meeting of the American Economic Association May, 1989  pp. ii-iv+vi+1-431+i-xxvi</a>, <a href="/stable/10.2307/i304561">No. 1 Mar., 1989  pp. 1-293+i-xxxix</a>, <a href="/stable/10.2307/i331337">No. 5 Dec., 1988  pp. 865-1164</a>, <a href="/stable/10.2307/i304595">No. 4 Sep., 1988  pp. 595-863+i-xxxviii</a>, <a href="/stable/10.2307/i331373">No. 3 Jun., 1988  pp. 307-594+i-xxviii</a>, <a href="/stable/10.2307/i331516">No. 2 Papers and Proceedings of the

processing https://www.jstor.org/journal/amereconrevi/decade/AXllYXI6WzE5NjAgVE8gMTk3MH0
[<a href="/stable/10.2307/i331420">No. 5 Dec., 1969  pp. 749-1030</a>, <a href="/stable/10.2307/i331447">No. 4 Part 1 Sep., 1969  pp. 457-748</a>, <a href="/stable/10.2307/i331435">No. 4 Part 2, Supplement, Surveys of National Economic Policy Issues and Policy Research Sep., 1969  pp. 1-118</a>, <a href="/stable/10.2307/i331387">No. 3 Jun., 1969  pp. 221-456</a>, <a href="/stable/10.2307/i331697">No. 2 Papers and Proceedings of the Eighty-first Annual Meeting of the American Economic Association May, 1969  pp. iii-v+vii-ix+1-629</a>, <a href="/stable/10.2307/i304609">No. 1 1969  pp. iii-x+1-220</a>, <a href="/stable/10.2307/i331466">No. 5 Dec., 1968  pp. 1159-1523</a>, <a href="/stable/10.2307/i331386">No. 5 Supplement, Part 2: Studies of the Structure of Economists' Salaries and Income Dec., 1968  pp. i-xxxv+1-153</a>, <a href="/stable/10.2307/i331496">No. 4 Sep., 1968  pp. 681-1157+i-xlvi</a>, <a

processing https://www.jstor.org/journal/amereconrevi/decade/AXllYXI6WzE5NDAgVE8gMTk1MH0
[<a href="/stable/10.2307/i331511">No. 6 Dec., 1949  pp. 1144-1400</a>, <a href="/stable/10.2307/i331413">No. 5 Sep., 1949  pp. 861-1141</a>, <a href="/stable/10.2307/i331398">No. 3 Jun., 1949  pp. 605-860</a>, <a href="/stable/10.2307/i331918">No. 3 Papers and Proceedings of the Sixty-first Annual Meeting of the American Economic Association May, 1949  pp. viii-x+2-537</a>, <a href="/stable/10.2307/i331426">No. 2 Mar., 1949  pp. iii-xiv+346-603</a>, <a href="/stable/10.2307/i332917">No. 1 The 1948 Directory of the American Economic Association Jan., 1949  pp. iii-xiv+iii-xi+1-343</a>, <a href="/stable/10.2307/i304621">No. 5 Dec., 1948  pp. 777-1022</a>, <a href="/stable/10.2307/i331244">No. 4 Sep., 1948  pp. 495-776</a>, <a href="/stable/10.2307/i331412">No. 3 Jun., 1948  pp. 245-493</a>, <a href="/stable/10.2307/i332662">No. 2 Papers and Proceedings of the Sixtieth Annual Meeting of the American 

processing https://www.jstor.org/journal/amereconrevi/decade/AXllYXI6WzE5MjAgVE8gMTkzMH0
[<a href="/stable/10.2307/i304589">No. 4 Dec., 1929  pp. 561-768</a>, <a href="/stable/10.2307/i331692">No. 3 Sep., 1929  pp. 357-560</a>, <a href="/stable/10.2307/i331350">No. 2 Jun., 1929  pp. 181-356</a>, <a href="/stable/10.2307/i331410">No. 1 Supplement, Papers and Proceedings of the Forty-first Annual Meeting of the American Economic Association Mar., 1929  pp. 1-284</a>, <a href="/stable/10.2307/i331396">No. 1 Mar., 1929  pp. i-x+1-180</a>, <a href="/stable/10.2307/i331409">No. 4 Dec., 1928  pp. 618-828</a>, <a href="/stable/10.2307/i331408">No. 3 Sep., 1928  pp. 406-427+429-448+450-467+469-616</a>, <a href="/stable/10.2307/i331356">No. 2 Supplement, Handbook of the American Economic Association 1928 Jun., 1928  pp. 1-138</a>, <a href="/stable/10.2307/i304588">No. 2 Jun., 1928  pp. 201-404</a>, <a href="/stable/10.2307/i331324">No. 1 Mar., 1928  pp. iii-ix+2-200</a>, <a href="/stable/10.2307

processing https://www.jstor.org/journal/amereconrevi/decade/AXllYXI6WzE5MTAgVE8gMTkyMH0
[<a href="/stable/10.2307/i331441">No. 4 Dec., 1919  pp. 681-946<span class="mls inline-block">
<span class="badge badge-border dark-green-color">
    FREE
</span>
</span></a>, <a href="/stable/10.2307/i304571">No. 3 Supplement, Handbook of the American Economic Association Sep., 1919  pp. 1-110<span class="mls inline-block">
<span class="badge badge-border dark-green-color">
    FREE
</span>
</span></a>, <a href="/stable/10.2307/i304570">No. 3 Sep., 1919  pp. 455-680<span class="mls inline-block">
<span class="badge badge-border dark-green-color">
    FREE
</span>
</span></a>, <a href="/stable/10.2307/i331691">No. 2 Jun., 1919  pp. 214-454<span class="mls inline-block">
<span class="badge badge-border dark-green-color">
    FREE
</span>
</span></a>, <a href="/stable/10.2307/i331455">No. 1 Supplement, Papers and Proceedings of the Thirty-First Annual Meeting of the American Economic Association Mar

## Handling allIsue

We want to create a csv file, or xlsx file, that has the following column names


In [10]:
colNames = [
    "Source URL",
    "Title",
    "Long Volume",
    "Issue Vol.",
    "Issue No.",
    "Month",
    "Year",
    "Abstract",
    "Full Author",
    "Author 1",
    "Author 2",
    "Author 3",
    "Author 4",
    "Author 5",
    "Author 6",
    "Author 7",
    "Author 8"
]

I stored the resuls of issues into another file

In [11]:
with open('outputfile') as jsonf:
    neededIssues = json.load(jsonf)

In [12]:
def parseIssue(t):
    separated = t.split(',')
    vol = separated[0].strip()
    no = separated[1].strip()
    month = separated[2].strip()
    year = separated[3].strip()
    return t, vol, no, month, year

## \s+and\s+|,(?!\s*Jr\.)\s*
def parseAuthor(a):
    
    authors = re.split(r'\s+and\s+|,(?!\s*Jr\.)\s*', a)
    split_authors = [e.strip() for e in authors]
    
    if len(split_authors) < 7:
        diff = 7 - len(split_authors)
        split_authors += [''] * diff
    return [a] + split_authors

In [13]:
temp1 = []
counter = 1
for issue in neededIssues:
    url = CONFIG.DOMAIN_URL + issue['url']
    issueBS = getBSFromURL(url)
    
    issue_name = issueBS.select('div.issue')[0].text.strip()
    print(f"Processing : {counter}/{len(neededIssues)} \t {issue_name} ...")
    articlesWithAuthor = issueBS.select('div.contrib')
    
    for article in articlesWithAuthor: 
        article_element = article.parent
        article_title = article_element.find('a').text.strip()
        article_url = CONFIG.DOMAIN_URL + article_element.find('a')['href']
        article_authors = parseAuthor( article.text.strip() )
        full_vol, vol, no, month, year = parseIssue(issue_name)
        temp1.append([
            article_url,
            article_title,
            full_vol, vol, no, month, year,
            ''
        ] + 
        article_authors)
        print("Article Name  :  " + article_title)
    print("")
    ImNotRobot()
    counter += 1 

Processing : 1/101 	 Vol. 88, No. 5, Dec., 1998 ...
Article Name  :  The Value of Weather Information Services for Nineteenth-Century Great Lakes Shipping
Article Name  :  The Rise and Fall of Bank Control in the United States: 1890-1939
Article Name  :  Winners and Losers in Russia's Economic Transition
Article Name  :  Unemployment and the Social Safety Net during Transitions to a Market Economy: Evidence from the Czech and Slovak Republics
Article Name  :  Federalism and the Soft Budget Constraint
Article Name  :  Interest-Group Competition and the Organization of Congress: Theory and Evidence from Financial Services' Political Action Committees
Article Name  :  Toward an Economic Theory of Leadership: Leading by Example
Article Name  :  Ambiguity Aversion and Incompleteness of Contractual Form
Article Name  :  Anticompetitive Vertical Integration by a Dominant Firm
Article Name  :  Patent Litigation as an Information-Transmission Mechanism
Article Name  :  The Home Market, Trade, a

Processing : 5/101 	 Vol. 88, No. 1, Mar., 1998 ...
Article Name  :  A Vision of the Growth Process
Article Name  :  Competition between Private and Public Schools, Vouchers, and Peer-Group Effects
Article Name  :  The Social Selection of Flexible and Rigid Agents
Article Name  :  Optimal Income Taxation: An Example with a U-Shaped Pattern of Optimal Marginal Tax Rates
Article Name  :  Endogenously Chosen Boards of Directors and Their Monitoring of the CEO
Article Name  :  The Simple Economics of Easter Island: A Ricardo-Malthus Model of Renewable Resource Use
Article Name  :  Sources of Inefficiency in a Representative Democracy: A Dynamic Analysis
Article Name  :  Altruists, Egoists, and Hooligans in a Local Interaction Model
Article Name  :  When Does It Take a Nixon to Go to China?
Article Name  :  What Price Coordination? The Efficiency-Enhancing Effect of Auctioning the Right to Play
Article Name  :  The International Ramifications of Tax Reforms: Supply-Side Economics in a Globa

Processing : 11/101 	 Vol. 87, No. 1, Mar., 1997 ...
Article Name  :  Trade Policy and Economic Development: How We Learn
Article Name  :  The Macroeconomic Effects of War Finance in the United States: World War II and the Korean War
Article Name  :  True Measures of GDP and Convergence
Article Name  :  Replicating Ricardian Equivalence Tests with Simulated Series
Article Name  :  Measuring the Liquidity Effect
Article Name  :  Optimal Inflation Targets, "Conservative" Central Banks, and Linear Inflation Contracts
Article Name  :  Aggregate Employment Dynamics: Building from Microeconomic Evidence
Article Name  :  Capital-Account Liberalization as a Signal
Article Name  :  Excess Volatility and Closed-End Funds
Article Name  :  The International Diversification Puzzle Is Worse Than You Think
Article Name  :  An Asset Allocation Puzzle
Article Name  :  The Consumption Smoothing Benefits of Unemployment Insurance
Article Name  :  Welfare-Improving Health Expenditure Subsidies
Article Nam

Processing : 16/101 	 Vol. 86, No. 1, Mar., 1996 ...
Article Name  :  Economics, Values, and Health Care Reform
Article Name  :  Mortgage Lending in Boston: Interpreting HMDA Data
Article Name  :  Productivity and the Density of Economic Activity
Article Name  :  Real-Business-Cycle Models and the Forecastable Movements in Output, Hours, and Consumption
Article Name  :  Money and Output: A Test of Reverse Causation
Article Name  :  Business Cycles and Labor-Market Search
Article Name  :  The Spirit of Capitalism and Stock-Market Prices
Article Name  :  Income Risk, Borrowing Constraints, and Portfolio Choice
Article Name  :  The LeChatelier Principle
Article Name  :  Auctions Versus Negotiations
Article Name  :  Organizational Design and Technology Choice under Intrafirm Bargaining
Article Name  :  Divisionalization, Franchising, and Divestiture Incentives in Oligopoly
Article Name  :  Strategic Trade Policies with Endogenous Mode of Competition
Article Name  :  Understanding the Costs

Processing : 21/101 	 Vol. 85, No. 1, Mar., 1995 ...
Article Name  :  Rationality and Social Choice
Article Name  :  Incumbent Behavior: Vote-Seeking, Tax-Setting, and Yardstick Competition
Article Name  :  Altruism, the Samaritan's Dilemma, and Government Transfer Policy
Article Name  :  On the Evolution of Altruistic Ethical Rules for Siblings
Article Name  :  Cartel Quotas Under Majority Rule
Article Name  :  Capital Mobility in Neoclassical Models of Growth
Article Name  :  Early Development
Article Name  :  The Transition from Barter to Fiat Money
Article Name  :  Optimal Contracts for Central Bankers
Article Name  :  Tastes and Technology in a Two-Country Model of the Business Cycle: Explaining International Comovements
Article Name  :  Union Contracts and the Life-Cycle/Permanent-Income Hypothesis
Article Name  :  Exchange Rates and Fundamentals: Evidence on Long-Horizon Predictability
Article Name  :  Monetary Policy Trade-offs and the Correlation between Nominal Interest Rates

...

Processing : 86/101 	 Vol. 72, No. 4, Sep., 1982 ...
Article Name  :  The Decline in Black Teenage Employment: 1950-70
Article Name  :  Applied Fairness Theory and Rationing Policy
Article Name  :  Towards an Understanding of Market Processes: Individual Expectations, Learning, and Convergence to Rational Expectations Equilibrium
Article Name  :  Welfare Measurement: A Synthesis
Article Name  :  The General Validity of the Heckscher-Ohlin Theorem
Article Name  :  Sharecropping and the Interlinking of Agrarian Markets
Article Name  :  The Importance of Lifetime Jobs in the U.S. Economy
Article Name  :  Accumulation of Property by Southern Blacks Before World War I
Article Name  :  The Price of Oil and World Inflation and Recession
Article Name  :  Obsolescence of Knowledge and Career Publication Profiles: Some Evidence of Differences among Fields in Costs of Interrupted Careers
Article Name  :  Inventories, Layoffs, and the Short-Run Demand for Labor
Article Name  :  Does Anticipated Ag

Processing : 89/101 	 Vol. 72, No. 1, Mar., 1982 ...
Article Name  :  Contestable Markets: An Uprising in the Theory of Industry Structure
Article Name  :  The Theory of Trade in Middle Products
Article Name  :  Capital Fixity, Innovations, and Long-Term Contracting: An Intertemporal Economic Theory of Regulation
Article Name  :  Barriers to Entry
Article Name  :  Competitive Market Institutions: Double Auctions vs. Sealed Bid-Offer Auctions
Article Name  :  Efficient Rules in Highway Safety and Sports Activity
Article Name  :  An Economic Model of the Brain Drain
Article Name  :  Social Insurance and Consumption: An Empirical Inquiry
Article Name  :  The Schumpeterian Tradeoff Revisited
Article Name  :  Veteran Status as a Screening Device
Article Name  :  Inflation Expectations and the Demand for Housing
Article Name  :  Inflation, Housing Costs, and the Consumer Price Index
Article Name  :  Valuing Public Goods: A Comparison of Survey and Hedonic Approaches
Article Name  :  Stochast

Processing : 94/101 	 Vol. 71, No. 2, May, 1981 ...
Article Name  :  Economics and Political Economy
Article Name  :  Inventories and the Structure of Macro Models
Article Name  :  Investment in Finished Goods Inventories: An Analysis of Adjustment Speeds
Article Name  :  Merchant Wholesaler Inventory Investment and the Cost of Capital
Article Name  :  Capital-Labor Conflict and the Productivity Slowdown
Article Name  :  Industrial Conflict and its Implications for Productivity Growth
Article Name  :  A Conflict Theory Approach to Inflation in the Postwar U.S. Economy
Article Name  :  Capacity Utilization Measures: Underlying Economic Theory and an Alternative Approach
Article Name  :  Stochastic Equilibrium and Capacity Utilization
Article Name  :  Long-Run Changes in the Workweek of Fixed Capital
Article Name  :  Slack Capacity: Productive or Wasteful?
Article Name  :  Female Labor Supply in the Context of Inflation
Article Name  :  A Times-Series Analysis of Women's Labor Force Part

Processing : 97/101 	 Vol. 70, No. 4, Sep., 1980 ...
Article Name  :  The Slowing Down of the Engine of Growth
Article Name  :  The Dynamics of Spot and Forward Prices in an Efficient Foreign Exchange Market with Rational Expectations
Article Name  :  Experiments with a Decentralized Mechanism for Public Good Decisions
Article Name  :  A Model of the Firm as a Stockholder-Employee Cooperative Game
Article Name  :  Tariffs, Nontraded Goods, and the Optimal Stabilization Policy
Article Name  :  Unemployment, the Allocation of Labor, and Optimal Government Intervention
Article Name  :  Fiscal Policies, Inflation, and Capital Formation
Article Name  :  A Model of Sales
Article Name  :  An Alternative Test of the Capital Asset Pricing Model
Article Name  :  Explaining the Relative Efficiency of Slave Agriculture in the Antebellum South: Reply
Article Name  :  On the Monopolistic Provision of Excludable Public Goods
Article Name  :  Taxation, Human Capital, and Uncertainty
Article Name  :  E

Processing : 100/101 	 Vol. 70, No. 1, Mar., 1980 ...
Article Name  :  On Theories of Unemployment
Article Name  :  Inflationary Expectations, Economic Activity, Taxes, and Interest Rates
Article Name  :  Oligopoly and Competition in Large Markets
Article Name  :  Price Inflation, Portfolio Choice, and Nominal Interest Rates
Article Name  :  Forecasting the Market for New Ph.D. Economists
Article Name  :  The Substitution Bias of the Laspeyres Price Index: An Analysis Using Estimated Cost-of-Living Indexes
Article Name  :  The Changing Cyclical Behavior of Wages and Prices: 1890-1976
Article Name  :  Family Size and the Distribution of Real Per Capita Income
Article Name  :  Sources of Quality Change in Labor Input
Article Name  :  Efficient Foreign Exchange Markets and the Monetary Approach to Exchange-Rate Determination
Article Name  :  Metzler on Classical Interest Theory
Article Name  :  Inflation Expectations and Money Growth in the United States
Article Name  :  Measuring Technol

In [14]:
issues_df = pd.DataFrame(temp1)
issues_df['scraped'] = [0]*len(issues_df.index)
issues_df.to_csv(f'{CONFIG.JOURNAL_NAME}_author_fixed.csv')

In [8]:
temp1

[['https://www.jstor.org/stable/116860',
  'The Value of Weather Information Services for Nineteenth-Century Great Lakes Shipping',
  'Vol. 88, No. 5, Dec., 1998',
  'Vol. 88',
  'No. 5',
  'Dec.',
  '1998',
  '',
  'Erik D. Craft',
  '',
  '',
  '',
  '',
  '',
  ''],
 ['https://www.jstor.org/stable/116861',
  'The Rise and Fall of Bank Control in the United States: 1890-1939',
  'Vol. 88, No. 5, Dec., 1998',
  'Vol. 88',
  'No. 5',
  'Dec.',
  '1998',
  '',
  'Miguel Cantillo Simon',
  '',
  '',
  '',
  '',
  '',
  ''],
 ['https://www.jstor.org/stable/116862',
  "Winners and Losers in Russia's Economic Transition",
  'Vol. 88, No. 5, Dec., 1998',
  'Vol. 88',
  'No. 5',
  'Dec.',
  '1998',
  '',
  'Elizabeth Brainerd',
  '',
  '',
  '',
  '',
  '',
  ''],
 ['https://www.jstor.org/stable/116863',
  'Unemployment and the Social Safety Net during Transitions to a Market Economy: Evidence from the Czech and Slovak Republics',
  'Vol. 88, No. 5, Dec., 1998',
  'Vol. 88',
  'No. 5',
  'Dec

In [51]:
from itertools import islice   
for index, article in islice(not_yet_scraped_articles_df.iterrows(),10):
    print(index)
    
    url = article['Source URL']
    print (url)
    
    getBs

0
https://www.jstor.org/stable/116860
2
https://www.jstor.org/stable/116862
3
https://www.jstor.org/stable/116863
4
https://www.jstor.org/stable/116864
5
https://www.jstor.org/stable/116865
6
https://www.jstor.org/stable/116866
7
https://www.jstor.org/stable/116867
8
https://www.jstor.org/stable/116868
9
https://www.jstor.org/stable/116869
10
https://www.jstor.org/stable/116870


In [6]:
parseAuthor('Eric B. Rasmusen, J. Mark Ramseyer and John S. Wiley, Jr.')

['Eric B. Rasmusen, J. Mark Ramseyer and John S. Wiley, Jr.',
 'Eric B. Rasmusen',
 'J. Mark Ramseyer',
 'John S. Wiley, Jr.',
 '',
 '',
 '',
 '']

In [6]:
def signal_handler(sig, frame):
    print("Scraping Quit")
    
    sys.exit(0)
    
signal.signal(signal.SIGINT, signal_handler)

running
running
running
running
running
running
running
running
running
running
running
running
yeeee


SystemExit: 0

In [57]:
not_yet_scraped_articles_df.iterrows()

<generator object DataFrame.iterrows at 0x7fd0f6329350>